In [15]:
# import
%pylab inline
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

Populating the interactive namespace from numpy and matplotlib


## Question 1

In [24]:
top_uni_json_link = "https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508402735534"
r = requests.get(top_uni_json_link)

In [90]:
def retrieve_people_number(url):
    
    url_complete = "https://www.topuniversities.com/" + url
    r = requests.get(url_complete)
    
    soup = BeautifulSoup(r.text, 'html.parser')
    
    numbers = None
    try:
        numbers = soup.find('div', {'id':'academic-data-carousel'}).find_all('div', class_='number')
    except:
        print("error at : " + url)
        return None
    
    numbers = list(map(lambda x: int(x.contents[0].replace('\n', '').replace(',', '')), numbers)) 
    if(len(numbers) == 4):
        return numbers
    else:
        print("wrong number of elements at : " + url)
        h3s = soup.find('div', {'id':'academic-data-carousel'}).find_all('h3')
        h3s = list(map(lambda x: (x.contents[0].replace('\n', '').replace(',', '')), h3s))
        original = ['Number of academic faculty staff','International','Number of students','Number of international students']
                
        result = [np.nan, np.nan, np.nan, np.nan]
        for h in range(0, len(h3s)):
            idx = original.index(h3s[h])
            result[idx] = numbers[h]
        
        return result

In [89]:
top_uni_df = pd.DataFrame(r.json()['data']).head(200)

missing_datas = pd.DataFrame( columns=['fac_members','inter_fac_members','students','inter_students'])
for index, row in top_uni_df.iterrows(): 
    tmp = retrieve_people_number(row.url)
    print(str(index) + " " + row.url + " : " + str(tmp))
    if(tmp != None):
        missing_datas.loc[index] = tmp
    else: 
        missing_datas.loc[index] = [np.nan, np.nan, np.nan, np.nan]


0 /universities/massachusetts-institute-technology-mit : [2982, 1679, 11067, 3717]
1 /universities/stanford-university : [4285, 2042, 15878, 3611]
2 /universities/harvard-university : [4350, 1311, 22429, 5266]
3 /universities/california-institute-technology-caltech : [953, 350, 2255, 647]
4 /universities/university-cambridge : [5490, 2278, 18770, 6699]
5 /universities/university-oxford : [6750, 2964, 19720, 7353]
6 /universities/ucl-university-college-london : [6345, 2554, 31080, 14854]
7 /universities/imperial-college-london : [3930, 2071, 16090, 8746]
8 /universities/university-chicago : [2449, 635, 13557, 3379]
9 /universities/eth-zurich-swiss-federal-institute-technology : [2477, 1886, 19815, 7563]
10 /universities/nanyang-technological-university-singapore-ntu : [4338, 2993, 25738, 7251]
11 /universities/ecole-polytechnique-f%C3%A9d%C3%A9rale-de-lausanne-epfl : [1695, 1300, 10343, 5896]
12 /universities/princeton-university : [1007, 246, 8069, 1793]
13 /universities/cornell-univer

117 /universities/university-california-davis : [3843, 549, 34244, 4529]
118 /universities/aarhus-university : [2316, 602, 26226, 3762]
119 /universities/humboldt-universit%C3%A4t-zu-berlin : [1966, 407, 33198, 4805]
120 /universities/universidade-de-s%C3%A3o-paulo : [5582, 279, 65711, 2086]
121 /universities/universidad-nacional-aut%C3%B3noma-de-m%C3%A9xico-unam : [15073, 949, 141939, 2173]
122 /universities/hokkaido-university : [2605, 210, 17552, 1347]
123 /universities/wageningen-university : [2277, 323, 12695, 3613]
124 /universities/freie-universitaet-berlin : [1375, 305, 32912, 5735]
125 /universities/ghent-university : [2520, 482, 35968, 3766]
126 /universities/queen-mary-university-london : [1885, 801, 16135, 6806]
127 /universities/kyushu-university : [2858, 246, 18660, 1787]
128 /universities/university-maryland-college-park : [3920, 473, 35504, 4683]
129 /universities/universit%C3%A9-de-montr%C3%A9al : [2961, 1037, 39112, 9688]
130 /universities/universit%C3%A9-pierre-et-ma

In [106]:
top_uni_df_complete = pd.concat([top_uni_df, missing_datas], axis=1)

top_uni_df_complete['ratio_fac_students'] = top_uni_df_complete['fac_members'].values / top_uni_df_complete['students']
top_uni_df_complete['ratio_inter_students'] = top_uni_df_complete['inter_students'].values / top_uni_df_complete['students']

top_uni_df_complete.drop(['cc','core_id', 'guide','logo','nid','score','stars','url'], axis=1, inplace=True)
top_uni_df_complete.head()

,country,rank_display,region,title,fac_members,inter_fac_members,students,inter_students,ratio_fac_students,ratio_inter_students
0,United States,1,North America,Massachusetts Institute of Technology (MIT),2982,1679,11067,3717,0.26945,0.335863
1,United States,2,North America,Stanford University,4285,2042,15878,3611,0.26987,0.227422
2,United States,3,North America,Harvard University,4350,1311,22429,5266,0.193945,0.234785
3,United States,4,North America,California Institute of Technology (Caltech),953,350,2255,647,0.422616,0.286918
4,United Kingdom,5,Europe,University of Cambridge,5490,2278,18770,6699,0.292488,0.356899


In [107]:
top_uni_df_complete.sort_values('ratio_fac_students', ascending=False).head()

,country,rank_display,region,title,fac_members,inter_fac_members,students,inter_students,ratio_fac_students,ratio_inter_students
3,United States,4,North America,California Institute of Technology (Caltech),953,350,2255,647,0.422616,0.286918
15,United States,16,North America,Yale University,4940,1708,12402,2469,0.398323,0.199081
5,United Kingdom,6,Europe,University of Oxford,6750,2964,19720,7353,0.342292,0.37287
4,United Kingdom,5,Europe,University of Cambridge,5490,2278,18770,6699,0.292488,0.356899
16,United States,17,North America,Johns Hopkins University,4462,1061,16146,4105,0.276353,0.254243


In [108]:
top_uni_df_complete.sort_values('ratio_inter_students', ascending=False).head()

,country,rank_display,region,title,fac_members,inter_fac_members,students,inter_students,ratio_fac_students,ratio_inter_students
34,United Kingdom,35,Europe,London School of Economics and Political Scien...,1088,687,9760,6748,0.111475,0.691393
11,Switzerland,12,Europe,Ecole Polytechnique Fédérale de Lausanne (EPFL),1695,1300,10343,5896,0.163879,0.570047
7,United Kingdom,8,Europe,Imperial College London,3930,2071,16090,8746,0.244251,0.543567
198,Netherlands,200,Europe,Maastricht University,1277,502,16385,8234,0.0779371,0.502533
47,United States,=47,North America,Carnegie Mellon University,1342,425,13356,6385,0.100479,0.478062


In [128]:
top_uni_agg_region = top_uni_df_complete[['region','fac_members','inter_fac_members','students','inter_students']].groupby(by = ['region']).agg(sum)

top_uni_agg_region['ratio_fac_students'] = top_uni_agg_region['fac_members'].values / top_uni_agg_region['students']
top_uni_agg_region['ratio_inter_students'] = top_uni_agg_region['inter_students'].values / top_uni_agg_region['students']

top_uni_agg_region

,fac_members,inter_fac_members,students,inter_students,ratio_fac_students,ratio_inter_students
region,,,,,,
Africa,1733.0,379,19593.0,3325.0,0.088450,0.169703
Asia,106734.0,25462,807003.0,110100.0,0.132260,0.136431
Europe,218358.0,67598,1957251.0,449364.0,0.111564,0.229589
Latin America,43126.0,5521,396902.0,34737.0,0.108657,0.087520
North America,182123.0,43836,1546353.0,292116.0,0.117776,0.188906
Oceania,25347.0,12786,350167.0,118798.0,0.072385,0.339261


In [129]:
top_uni_agg_country = top_uni_df_complete[['country','fac_members','inter_fac_members','students','inter_students']].groupby(by = ['country']).agg(sum)

top_uni_agg_country['ratio_fac_students'] = top_uni_agg_country['fac_members'].values / top_uni_agg_country['students']
top_uni_agg_country['ratio_inter_students'] = top_uni_agg_country['inter_students'].values / top_uni_agg_country['students']

top_uni_agg_country

,fac_members,inter_fac_members,students,inter_students,ratio_fac_students,ratio_inter_students
country,,,,,,
Argentina,16421.0,3165,122301.0,27109.0,0.134267,0.221658
Australia,22034.0,11382,301994.0,106359.0,0.072962,0.352189
Austria,4117.0,1572,63446.0,19667.0,0.064890,0.309980
Belgium,8046.0,2187,115067.0,17013.0,0.069924,0.147853
Brazil,7550.0,388,92283.0,3052.0,0.081814,0.033072
Canada,29317.0,10734,281514.0,73239.0,0.104140,0.260161
Chile,2260.0,198,27003.0,991.0,0.083694,0.036700
China,27220.0,6720,235898.0,26833.0,0.115389,0.113748
Denmark,11916.0,3904,67223.0,9543.0,0.177261,0.141960


## Question 2

In [209]:
times_high_json_link = "https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json"
r = requests.get(times_high_json_link)

In [210]:
times_high_df = pd.DataFrame(r.json()['data']).head(200)

times_high_df = times_high_df[['location','aliases','name','rank','stats_number_students','stats_pc_intl_students','stats_student_staff_ratio']]
times_high_df.head()

,location,aliases,name,rank,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio
0,United Kingdom,University of Oxford,University of Oxford,1,"20,409",38%,11.2
1,United Kingdom,University of Cambridge,University of Cambridge,2,"18,389",35%,10.9
2,United States,California Institute of Technology caltech,California Institute of Technology,=3,"2,209",27%,6.5
3,United States,Stanford University,Stanford University,=3,"15,845",22%,7.5
4,United States,Massachusetts Institute of Technology,Massachusetts Institute of Technology,5,"11,177",34%,8.7


In [211]:
times_high_df['ratio_inter_student'] = times_high_df['stats_pc_intl_students'].str.replace('%','').astype(int) / 100.0
times_high_df['stats_student_staff_ratio'] = 1.0 / times_high_df['stats_student_staff_ratio'].astype(float)
times_high_df.head()

,location,aliases,name,rank,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,ratio_inter_student
0,United Kingdom,University of Oxford,University of Oxford,1,"20,409",38%,0.089286,0.38
1,United Kingdom,University of Cambridge,University of Cambridge,2,"18,389",35%,0.091743,0.35
2,United States,California Institute of Technology caltech,California Institute of Technology,=3,"2,209",27%,0.153846,0.27
3,United States,Stanford University,Stanford University,=3,"15,845",22%,0.133333,0.22
4,United States,Massachusetts Institute of Technology,Massachusetts Institute of Technology,5,"11,177",34%,0.114943,0.34


In [212]:
times_high_df['students'] = times_high_df['stats_number_students'].astype(str).str.replace(',','').astype(int)
times_high_df['inter_students'] = np.round(times_high_df['students'] * times_high_df['ratio_inter_student']).astype(int)
times_high_df['fac_members'] = np.round(times_high_df['students'] * times_high_df['stats_student_staff_ratio']).astype(int)
times_high_df

,location,aliases,name,rank,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,ratio_inter_student,students,inter_students,fac_members
0,United Kingdom,University of Oxford,University of Oxford,1,"20,409",38%,0.089286,0.38,20409,7755,1822
1,United Kingdom,University of Cambridge,University of Cambridge,2,"18,389",35%,0.091743,0.35,18389,6436,1687
2,United States,California Institute of Technology caltech,California Institute of Technology,=3,"2,209",27%,0.153846,0.27,2209,596,340
3,United States,Stanford University,Stanford University,=3,"15,845",22%,0.133333,0.22,15845,3486,2113
4,United States,Massachusetts Institute of Technology,Massachusetts Institute of Technology,5,"11,177",34%,0.114943,0.34,11177,3800,1285
5,United States,Harvard University,Harvard University,6,"20,326",26%,0.112360,0.26,20326,5285,2284
6,United States,Princeton University,Princeton University,7,"7,955",24%,0.120482,0.24,7955,1909,958
7,United Kingdom,Imperial College London,Imperial College London,8,"15,857",55%,0.087719,0.55,15857,8721,1391
8,United States,University of Chicago,University of Chicago,9,"13,525",25%,0.161290,0.25,13525,3381,2181
9,Switzerland,ETH Zurich – Swiss Federal Institute of Techno...,ETH Zurich – Swiss Federal Institute of Techno...,=10,"19,233",38%,0.068493,0.38,19233,7309,1317


In [224]:
times_high_df_complete = times_high_df.drop(['stats_pc_intl_students', 'aliases', 'stats_number_students'], axis=1)
times_high_df_complete.rename(columns = {'stats_student_staff_ratio': 'ratio_fac_students', 'location': 'country','name': 'title'}, inplace=True)

In [225]:
times_high_agg_country = times_high_df_complete[['country','fac_members','students','inter_students']].groupby(by = ['country']).agg(sum)

times_high_agg_country['ratio_fac_students'] = times_high_agg_country['fac_members'].values / times_high_agg_country['students']
times_high_agg_country['ratio_inter_students'] = times_high_agg_country['inter_students'].values / times_high_agg_country['students']

times_high_agg_country

,fac_members,students,inter_students,ratio_fac_students,ratio_inter_students
country,,,,,
Australia,9940,268630,83812,0.037003,0.311998
Austria,1701,35375,9198,0.048085,0.260014
Belgium,3428,130115,23769,0.026346,0.182677
Canada,13240,249401,55906,0.053087,0.224161
China,21390,253911,22883,0.084242,0.090122
Denmark,10598,64110,9327,0.165310,0.145484
Finland,2022,35276,3901,0.057319,0.110585
France,7437,99378,18975,0.074835,0.190938
Germany,15355,586364,87490,0.026187,0.149208


## Question 3

In [257]:
top_uni_bef_merge = top_uni_df_complete.copy()
top_uni_bef_merge['title'] = top_uni_bef_merge['title'].str.lower(). \
    str.replace('é','e'). \
    str.replace(r'\(.*\)', ''). \
    str.replace('the',''). \
    str.replace('of',''). \
    str.replace('at',''). \
    str.replace(r',|\.',''). \
    str.replace(' ',''). \
    str.replace('\'','’')

times_high_bef_merge = times_high_df_complete.copy()
times_high_bef_merge['title'] = times_high_bef_merge['title'].str.lower(). \
    str.replace('é','e'). \
    str.replace(r'\(.*\)', ''). \
    str.replace('the',''). \
    str.replace('of',''). \
    str.replace('at',''). \
    str.replace(r',|\.',''). \
    str.replace(' ',''). \
    str.replace('\'','’')

inner_title = top_uni_bef_merge.merge(times_high_bef_merge, on='title', how='inner').sort_values('title')['title']
outer_title = top_uni_bef_merge.merge(times_high_bef_merge, on='title', how='outer').sort_values('title')['title']

merge_df = top_uni_bef_merge.merge(times_high_bef_merge, on='title', how='inner').sort_values('title')
merge_df

,country_x,rank_display,region,title,fac_members_x,inter_fac_members,students_x,inter_students_x,ratio_fac_students_x,ratio_inter_students,country_y,rank,ratio_fac_students_y,ratio_inter_student,students_y,inter_students_y,fac_members_y
110,Finland,=137,Europe,aaltouniversity,1257,370,12147,1831,0.103482,0.150737,Finland,190,0.049505,0.20,12744,2549,631
102,Denmark,119,Europe,aarhusuniversity,2316,602,26226,3762,0.0883093,0.143445,Denmark,=109,0.069930,0.12,25167,3020,1760
17,Australia,20,Oceania,australiannionaluniversity,1600,927,14442,5551,0.110788,0.384365,Australia,48,0.051813,0.35,15986,5595,828
70,United States,81,North America,bostonuniversity,3157,379,25662,7041,0.123022,0.274375,United States,=70,0.116279,0.25,24833,6208,2888
48,United States,53,North America,brownuniversity,1303,379,9251,1825,0.14085,0.197276,United States,=50,0.093458,0.20,8898,1780,832
3,United States,4,North America,californiainstitutetechnology,953,350,2255,647,0.422616,0.286918,United States,=3,0.153846,0.27,2209,596,340
111,United Kingdom,=137,Europe,cardiffuniversity,2755,705,24565,6090,0.112151,0.247914,United Kingdom,=162,0.074074,0.26,24186,6288,1792
43,United States,=47,North America,carnegiemellonuniversity,1342,425,13356,6385,0.100479,0.478062,United States,24,0.074074,0.45,12676,5704,939
41,Hong Kong,46,Asia,chineseuniversityhongkong,2208,1074,18037,4824,0.122415,0.26745,Hong Kong,58,0.057803,0.31,18072,5602,1045
44,Hong Kong,49,Asia,cityuniversityhongkong,1349,1027,9240,3273,0.145996,0.354221,Hong Kong,=119,0.089286,0.35,9240,3234,825


In [259]:
merge_df.corr()

,ratio_fac_students_y,ratio_inter_student,students_y,inter_students_y,fac_members_y
ratio_fac_students_y,1.000000,-0.052573,-0.376703,-0.363125,0.440064
ratio_inter_student,-0.052573,1.000000,-0.331599,0.487247,-0.314944
students_y,-0.376703,-0.331599,1.000000,0.555516,0.556060
inter_students_y,-0.363125,0.487247,0.555516,1.000000,0.179652
fac_members_y,0.440064,-0.314944,0.556060,0.179652,1.000000
